In [1]:
import numpy
import os
import xarray
from datetime import datetime, timedelta

In [2]:
MOD05_TPEV_FLODER = '/disk2/Data/MOD05_L2_day_GeoTiff'

In [3]:
def read_tpwv_site(modis_tpwv_tif, site_locs):
    modis_wv_ds = xarray.open_rasterio(modis_tpwv_tif)
    sites_v = []
    for site_loc in site_locs:
        lat_site = site_loc[0]
        lon_site = site_loc[1]
        site_ds = modis_wv_ds.interp(x=[lon_site], y=[lat_site], method="nearest", kwargs={"fill_value": "extrapolate"}) # linear
        wv_dn = numpy.array(site_ds.values)[0][0][0]
        if wv_dn == -9999:
            sites_v.append(numpy.NaN)
        else:
            data_v = wv_dn*0.001+0.0
            sites_v.append(data_v)
    return sites_v

In [4]:
tky_site_loc = (36.146, 137.423) # lat, lon
crk_site_loc = (38.201, 127.251) # lat, lon
gck_site_loc = (37.748, 127.162) # lat, lon
gdk_site_loc = (37.749, 127.149) # lat, lon
site_names = ['TKY','CRK', 'GCK', 'GDK']
site_locs = [tky_site_loc, crk_site_loc, gck_site_loc, gdk_site_loc]

In [5]:
years = ['2018', '2019']
year_days = [(3-len(str(day_n)))*'0' + str(day_n) for day_n in numpy.arange(1, 366, 1)]

site_record = []
for site_idx in range(len(site_names)):
    site_record.append([])

for year_str in years:
    year_first_day_date = datetime.strptime(year_str + '-01-01T00:00:00Z', "%Y-%m-%dT%H:%M:%SZ")
    for day_idx in range(len(year_days)):
        day_str = year_days[day_idx]
        data_filename = os.path.join(MOD05_TPEV_FLODER, year_str, 'MOD05_L2.A' + year_str + day_str + '.tif')

        this_day_date = year_first_day_date + timedelta(days=day_idx)
        this_day_time = this_day_date.strftime("%Y-%m-%dT%H:%M:%SZ")

        day_site_vs = read_tpwv_site(data_filename, site_locs)
        
        for site_idx in range(len(site_names)):
            record_item = [this_day_time, str(day_site_vs[site_idx])]
            site_record[site_idx].append(record_item)
            
wv_record = numpy.array(site_record)
for site_idx in range(len(site_names)):
    numpy.savetxt(os.path.join(MOD05_TPEV_FLODER, site_names[site_idx]+'_MOD05_TPWV'+'.csv'), wv_record[site_idx], delimiter=",", fmt='%s')